In [2]:
import datetime
import os

import numpy  as np
import pandas as pd

In [3]:
# Variables

pickleDir = '../Data/Pickle'
inputFile = '../Data/Fields/D11.csv'


In [4]:
print('input datas:',datetime.datetime.now())

frame = pd.read_csv(inputFile)

print('end input',datetime.datetime.now())
print(frame.tail())

input datas: 2016-05-25 17:30:13.358580
end input 2016-05-25 17:30:52.965288
          user_id  is_booking  dt0m  dt0y  hotel_cluster  countCnt  count
15015383  1198784           0     4  2015             26         2      1
15015384  1198784           0     4  2015             68         6      1
15015385  1198784           0     4  2015             95         1      1
15015386  1198784           0     8  2014             79         1      1
15015387  1198784           1     8  2014             79         1      1


In [7]:
grouped=frame.groupby(['user_id','is_booking']); print('user+book =',len(grouped))
print('grouped datas:',datetime.datetime.now())
print(len(grouped))

user+book = 1512327
grouped datas: 2016-05-25 17:44:25.459598
1512327


In [147]:
i        = -1
nLargest =  5; n = nLargest;

dcol = ['count','countCnt'] + ['n'+str(i) for i in range(5)] + ['cc'+str(i) for i in range(100)]
dcol = ['count','countCnt'] + ['n'+str(i) for i in range(n)]  + ['v'+str(i) for i in range(n)] 
def f (xgroup) :
    global i,n
    d00  = np.zeros((100));d00[xgroup['hotel_cluster']]+=xgroup['count']
    dsum = xgroup['countCnt'].sum()
    dcnt = xgroup['count'].sum()
    d05  = pd.Series(d00); 
    n05=d05[d05>0].nlargest(n).index.tolist()+[-1]*n; n05=n05[0:n];
    v05=d05[d05>0].nlargest(n).tolist()+[-1]*n;       v05=v05[0:n];
    ddd  = [dcnt,dsum] + n05 + v05 # + d00.tolist()
    #i   += 1
    #if (i%10000==0) : print('5 :',datetime.datetime.now()," i = ",i)

    
    return(pd.DataFrame(ddd,index=dcol).T)



In [148]:
def ff (transx,nn,filepickle) :
    trans0 = transx.query('is_booking==0')[['user_id','is_booking','count','countCnt']+ \
                                           ['n'+str(i) for i in range(nn)]+['v'+str(i) for i in range(nn)]]
    trans0.index = trans0['user_id']

    trans1 = transx.query('is_booking==1')[['user_id','is_booking','count','countCnt']+ \
                                           ['n'+str(i) for i in range(nn)]+['v'+str(i) for i in range(nn)]]
    trans1.index = trans1['user_id']

    trans01 = pd.concat([trans0,trans1],axis=1)

    trans01.columns = ['user_id','bo0','count0','countCnt0']+['b0n'+str(i) for i in range(nn)]+['b0v'+str(i) for i in range(nn)]+ \
                        ['h1','bo1','count1','countCnt1']+['b1n'+str(i) for i in range(nn)]+['b1v'+str(i) for i in range(nn)]
    trans01 = trans01[['user_id','count0','countCnt0','count1','countCnt1'] + \
                      ['b0n'+str(i) for i in range(nn)]+['b1n'+str(i) for i in range(nn)]+ \
                      ['b0v'+str(i) for i in range(nn)]+['b1v'+str(i) for i in range(nn)]]
    
    trans01.to_pickle(filepickle)
    
    return(trans01)


In [151]:
print(len(grouped))
print(frame['user_id'].min(),frame['user_id'].max())

ll = frame['user_id'].max()+1
l = [i for i in range(0,ll,10000)]
if (l[len(l)-1]!=ll) : l.append(ll)
mm = [(l[i],l[i+1]) for i in range(len(l)-1)]
#print(m)
#for i1,i2 in mm[-10:] : print((i2-i1))
print(len(mm),mm[0:5])

1512327
0 1198784
120 [(0, 10000), (10000, 20000), (20000, 30000), (30000, 40000), (40000, 50000)]


In [152]:
fres = []
i1,i2 = 10000,20000
for i1,i2 in mm :
    print('4 :',datetime.datetime.now(),' i1,i2=',i1,i2)
    cond = '('+str(i1)+'<=user_id)&(user_id<'+str(i2)+')'
    grouped100 = frame.query(cond).groupby(['user_id','is_booking']); 
    #print('user+book =',len(grouped100))
    #print('grouped (x in) datas:',datetime.datetime.now())
    i, n = -1, nLargest
    trans   = grouped100.apply(f);
    trans   = trans.reset_index()
    ffile   = os.path.join(pickleDir,'trans01-'+str(i1+1000000000)+'.pck')
    trans01 = ff(trans,n,ffile)
    fres.append(ffile)
    #print('6 :',datetime.datetime.now())
    
print('7 :',datetime.datetime.now())
print('fres=',len(fres))
trans01.head()

4 : 2016-05-25 20:32:36.461329  i1,i2= 0 10000


C:\Anaconda3-64\lib\site-packages\ipykernel\__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


4 : 2016-05-25 20:33:22.285353  i1,i2= 10000 20000
4 : 2016-05-25 20:34:15.759275  i1,i2= 20000 30000
4 : 2016-05-25 20:35:16.877032  i1,i2= 30000 40000
4 : 2016-05-25 20:36:33.070324  i1,i2= 40000 50000
4 : 2016-05-25 20:37:43.515027  i1,i2= 50000 60000
4 : 2016-05-25 20:38:50.388549  i1,i2= 60000 70000
4 : 2016-05-25 20:40:03.747865  i1,i2= 70000 80000
4 : 2016-05-25 20:41:13.654163  i1,i2= 80000 90000
4 : 2016-05-25 20:42:25.329281  i1,i2= 90000 100000
4 : 2016-05-25 20:43:37.268534  i1,i2= 100000 110000
4 : 2016-05-25 20:44:44.564925  i1,i2= 110000 120000
4 : 2016-05-25 20:45:53.422784  i1,i2= 120000 130000
4 : 2016-05-25 20:46:59.821477  i1,i2= 130000 140000
4 : 2016-05-25 20:48:11.207877  i1,i2= 140000 150000
4 : 2016-05-25 20:49:21.263340  i1,i2= 150000 160000
4 : 2016-05-25 20:50:36.491093  i1,i2= 160000 170000
4 : 2016-05-25 20:51:58.119528  i1,i2= 170000 180000
4 : 2016-05-25 20:53:25.635471  i1,i2= 180000 190000
4 : 2016-05-25 20:54:40.573877  i1,i2= 190000 200000
4 : 2016-0

,user_id,count0,countCnt0,count1,countCnt1,b0n0,b0n1,b0n2,b0n3,b0n4,...,b0v0,b0v1,b0v2,b0v3,b0v4,b1v0,b1v1,b1v2,b1v3,b1v4
user_id,,,,,,,,,,,,,,,,,,,,,
1190000,1190000.0,55.0,104.0,NaN,NaN,4.0,1.0,19.0,55.0,5.0,...,13.0,11.0,5.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN
1190001,1190001.0,5.0,6.0,1.0,2.0,9.0,41.0,42.0,-1.0,-1.0,...,2.0,2.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
1190003,1190003.0,2.0,2.0,1.0,1.0,25.0,55.0,-1.0,-1.0,-1.0,...,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
1190005,1190005.0,78.0,89.0,9.0,9.0,85.0,5.0,48.0,10.0,3.0,...,8.0,4.0,4.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0
1190006,1190006.0,160.0,204.0,3.0,3.0,45.0,46.0,98.0,4.0,19.0,...,9.0,6.0,6.0,4.0,4.0,2.0,1.0,-1.0,-1.0,-1.0


In [153]:
ff  = pd.read_pickle(fres[0])
for ifile in fres[1:len(fres)] :
    ff1 = pd.read_pickle(ifile)
    ff  = pd.concat([ff,ff1])
    del ff1
print(ff.shape)
ff.tail()

(916397, 25)


,user_id,count0,countCnt0,count1,countCnt1,b0n0,b0n1,b0n2,b0n3,b0n4,...,b0v0,b0v1,b0v2,b0v3,b0v4,b1v0,b1v1,b1v2,b1v3,b1v4
user_id,,,,,,,,,,,,,,,,,,,,,
1198780,1198780.0,2.0,2.0,1.0,1.0,21.0,80.0,-1.0,-1.0,-1.0,...,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
1198781,1198781.0,10.0,18.0,NaN,NaN,70.0,0.0,6.0,25.0,39.0,...,3.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1198782,1198782.0,3.0,7.0,NaN,NaN,37.0,55.0,77.0,-1.0,-1.0,...,1.0,1.0,1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN
1198783,1198783.0,23.0,33.0,4.0,4.0,23.0,39.0,5.0,6.0,15.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0
1198784,1198784.0,4.0,10.0,1.0,1.0,26.0,68.0,79.0,95.0,-1.0,...,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0


In [154]:
print(ff.shape)
ff.to_pickle(os.path.join(pickleDir,'trans01-user-id-all.pck'))
del ff

(916397, 25)


In [155]:
ff = pd.read_pickle(os.path.join(pickleDir,'trans01-user-id-all.pck'))
ff.shape

(916397, 25)

In [156]:
print('result(len,0+1,only 0) -->',ff.shape[0],ff[pd.notnull(ff.count1)].shape[0],ff[pd.isnull(ff.count1)].shape[0])

result(len,0+1,only 0) --> 916397 596662 319735


In [157]:
ff[ff.b0n0==ff.b1n0].shape[0]/ff.shape[0]

0.21921176084164395

In [158]:
print('count0 = (mean,max)',ff['count0'].mean(),ff['count0'].max())
print('count1 = (mean,max)',ff['count1'].mean(),ff['count1'].max())

count0 = (mean,max) 24.200748090185822 491.0
count1 = (mean,max) 3.3277031217003934 100.0


In [159]:
import matplotlib
matplotlib.style.use('ggplot')

In [160]:
ff[['count0','count1']].plot(x='count0',y='count1')